In [ ]:
%%writefile .env 
SUPABASE_URL = ""
SUPABASE_ANON_KEY = ""

In [1]:
!pip install python-dotenv # .env에 저장된 환경변수 로드해주는 패키지
!pip install supabase # 수파베이스 SDK

In [1]:
# 환경 변수 로드
from dotenv import load_dotenv
import os
load_dotenv(".env")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_ANON_KEY = os.getenv("SUPABASE_ANON_KEY")

In [3]:
# 수파베이스 접속
import supabase
sb = supabase.create_client(SUPABASE_URL, SUPABASE_ANON_KEY)

In [9]:
# 빈테이블 조회
sb.table('links').select('*').execute()
# '''select url from public.links;'''


APIResponse[TypeVar](data=[], count=None)

In [10]:
# 데이터 삽입
sb.table('links').insert([{'url': 'https://www.naver.com'},{'url': 'https://www.naver.com'},{'url': 'https://www.naver.com'}]).execute()

APIResponse[TypeVar](data=[{'id': 1, 'created_at': '2024-05-02T22:37:40.122463+00:00', 'url': 'https://www.naver.com'}, {'id': 2, 'created_at': '2024-05-02T22:37:40.122463+00:00', 'url': 'https://www.naver.com'}, {'id': 3, 'created_at': '2024-05-02T22:37:40.122463+00:00', 'url': 'https://www.naver.com'}], count=None)

In [17]:
# 데이터 다시 조회해서 데이터 프레임에 담기
import pandas as pd
data = sb.table('links').select('*').execute().data
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []


In [16]:
# 데이터 삭제 (id 값 확인할것)
sb.table('links').delete().eq('id', 1).execute()

APIResponse[TypeVar](data=[{'id': 3, 'created_at': '2024-05-02T22:37:40.122463+00:00', 'url': 'https://www.naver.com'}], count=None)

___
데이터베이스 준비 끝
___

###  더미 데이터 생성 & 삽입

In [ ]:
!pip install sentence-transformers

In [15]:
from sentence_transformers import SentenceTransformer
from supabase import create_client
import os, random, dotenv

# --- 설정 -------------------------------------------------------------
dotenv.load_dotenv()                       # .env 읽기
supabase = create_client(os.environ["SUPABASE_URL"],
                         os.environ["SUPABASE_ANON_KEY"])
model = SentenceTransformer("all-MiniLM-L6-v2")

dummy_texts = [
    "STATRACK 1기 AI엔지니어 양성과정",
    "맛집 추천 챗봇을 위한 레시피 임베딩",
    "한국 프로야구 2025 시즌 미디어 가이드",
    "초보자를 위한 Docker 입문서",
    "고양이를 위한 스마트 캣타워 개발기",
    "제주도 2박 3일 여행 코스",
    "MCP 서버 구축 자동화 튜토리얼",
    "SQL로 하는 시계열 데이터 예측",
    "n8n으로 만드는 Slack 알림 봇",
    "Python 데이터 클래스 활용법",
] * 2  # 20개

rows = []
for text in dummy_texts:
    vec = model.encode(text).tolist()       # numpy → list
    rows.append({"content": text, "embedding": vec})

# --- 삽입 -------------------------------------------------------------
inserted = supabase.table("embeddings").insert(rows).execute()
print(f"Inserted {len(inserted.data)} rows ✅")


Inserted 20 rows ✅


In [ ]:
%%writefile query_rpc.py

"""
$ python query_rpc.py "검색할 문장"
"""
import sys, os, dotenv
from sentence_transformers import SentenceTransformer
from supabase import create_client

dotenv.load_dotenv()
supabase = create_client(os.environ["SUPABASE_URL"],
                         os.environ["SUPABASE_ANON_KEY"])
model = SentenceTransformer("all-MiniLM-L6-v2")

query = sys.argv[1] if len(sys.argv) > 1 else "AI 엔지니어 과정"
qvec  = model.encode(query).tolist()

resp = supabase.rpc("match_embeddings", {
    "query_embedding": qvec,
    "match_threshold": 0.5,
    "match_count": 3
}).execute()

print(f"\n🔍  Query: {query}\n")
for r in resp.data:
    print(f"{r['similarity']:.3f}\t{r['content']}")
